**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-je9ai7ck
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-je9ai7ck
  Resolved https://github.com/Kaggle/learntools.git to commit 940f894c224b3737a597a80e9a792face28c6562
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268968 sha256=b8c6480f37a75ef8333e91b72f6972939b187bda0f78bd2adbb19b4b6e206b70
  Stored in directory: /tmp/pip-ephem-wheel-cache-qvzsug2w/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,d0dd5f20073d6174a456961088231dc46006a5c0,efce437d4bc2a79b12ac131e37789da8fe5e53f37d12c2...,2015-10-17 19:15:00+00:00,2015-10-17 19:30:00+00:00,780,2.2,1.703107e+10,1.703106e+10,7,6,...,0.0,10.20,Credit Card,None,41.926811,-87.642605,POINT (-87.6426052473 41.9268111821),41.953400,-87.646007,POINT (-87.6460070664 41.9534000435)
1,37682f189f404780d57f266556003849c5fec752,a02e1f355041b6350e4d79a4de1e650b266f5dc1ea40d1...,2015-10-12 23:45:00+00:00,2015-10-12 23:45:00+00:00,300,1.0,NaN,NaN,11,15,...,0.0,5.45,Cash,None,41.978830,-87.771167,POINT (-87.771166703 41.9788295262),41.954028,-87.763399,POINT (-87.7633990316 41.9540276487)
2,7add88253c78688a6a0920cb30adbb4944418601,e637e1afe0dab0e2d04c635335602caf1045d43e3741c2...,2015-10-16 15:45:00+00:00,2015-10-16 15:45:00+00:00,360,0.8,1.703108e+10,1.703184e+10,8,32,...,1.0,6.25,Cash,None,41.909496,-87.630964,POINT (-87.630963601 41.9094956686),41.871016,-87.631407,POINT (-87.6314065252 41.8710158803)
3,7d597ed8fa15502482d3c98b74fdc084785c7f14,a538132e78f04101a86f83f4697f6944ea8c97acef8eb3...,2015-10-11 02:15:00+00:00,2015-10-11 02:15:00+00:00,780,6.6,1.703103e+10,1.703107e+10,77,7,...,1.0,16.65,Cash,None,41.994442,-87.665225,POINT (-87.6652247765 41.9944422482),41.921855,-87.646211,POINT (-87.6462109769 41.9218549112)
4,053c4b3fe66ca85a76998502de77bbc34ddf4668,0c1aa5b32422ba0cc2082cd87c91d95f8ec088421d886a...,2015-10-11 01:30:00+00:00,2015-10-11 01:45:00+00:00,480,2.0,1.703105e+10,1.703104e+10,5,4,...,0.0,7.45,Cash,None,41.950546,-87.676182,POINT (-87.6761824959 41.95054569650001),41.966806,-87.694020,POINT (-87.6940200973 41.9668063463)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [5]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          avg(num_trips)
                          OVER (
                               order by trip_date
                               rows between 3 preceding and 3 following
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-26,92183.285714
1,2016-02-14,92247.142857
2,2016-03-29,75684.000000
3,2016-02-28,92816.428571
4,2016-01-10,82083.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [8]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        rank() over (
                                    partition by pickup_community_area
                                    order by trip_start_timestamp
                                    ) as trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,4.0,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
1,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
2,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:30:00+00:00,2
3,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
4,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [ ]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1)
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

In [11]:
# Lines below will give you a hint or solution code
#q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

```

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*